In [1]:
import pandas as pd
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
from nucleotide_transformer.pretrained import get_pretrained_model
from tqdm import tqdm
import sys
sys.path.append('/home/ztang/multitask_RNA/data_generation')
import utils
import h5py
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import glob
import os
os.environ['CUDA_VISIBLE_DEVIES'] = '0'

2023-06-28 13:58:07.945042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file = '/home/ztang/multitask_RNA/data/eclip/PUM2_K562_200.h5'

In [3]:
f = h5py.File(file,'r')

In [9]:
f['X_test']

<HDF5 dataset "X_test": shape (2900, 9, 200), type "<f4">

In [11]:
datalen = 200
max_len = datalen//6 + datalen%6 + 1
random_key = jax.random.PRNGKey(0)
parameters, forward_fn, tokenizer, config = get_pretrained_model(
                model_name='2B5_1000G',
                mixed_precision=False,
                embeddings_layers_to_save=(32,),
                attention_maps_to_save=(),
                max_positions=max_len
            )
forward_fn = hk.transform(forward_fn)

In [36]:
batch_size = 32
data = f
cls_dataset = {}
mean_dataset = {}

#create dataset for train,valid,test.
for label in ['test', 'valid','train']:
    #tokenization
    sequence = data['x_'+label][()]
    sequence = np.transpose(sequence,(0,2,1))
    sequence = utils.onehot_to_seq(sequence)
    target = data['y_'+label][()]
    token_out = tokenizer.batch_tokenize(sequence)
    token_id = [b[1] for b in token_out]
    seq_pair= np.squeeze(token_id)

    #pass through model for embeddings
    cls = []
    mean_embed = []
    for i in tqdm(range(0, len(seq_pair), batch_size)):
        seq_batch = jnp.asarray(seq_pair[i:i+batch_size,:],dtype=jnp.int32)
        outs = forward_fn.apply(parameters, random_key, seq_batch)
        embed = np.array(outs['embeddings_'+str(32)])
        cls.extend(embed[:,:1, :])
        embed = embed[:, 1:, :]  # removing CLS token
        mean_embed.extend(np.sum(embed, axis=1) / embed.shape[1])

    cls_dataset[label] = np.array(cls)
    mean_dataset[label] = np.array(mean_embed)

  3%|▎         | 10/319 [00:08<04:32,  1.13it/s]


KeyboardInterrupt: 

In [33]:
seq_batch

Array([[   3, 1657,  658,  634, 1714, 2607, 3311,  891, 3585, 3574, 2411,
        3847, 3960,  143, 1944, 3183, 3776,  942, 3534, 4091, 2283, 2651,
        3473,  995, 2095, 1903, 2421, 1411, 1018, 2871, 3713, 3651, 2476,
        3895, 4103, 4102]], dtype=int32)

In [35]:
embed.shape

(1, 35, 2560)

In [21]:
expect_len = 2560
i = 0
for seq in cls:
    
    if len(seq) != expect_len:
        print(len(seq))
        break
    i+=1

TypeError: object of type 'numpy.float32' has no len()

In [22]:
i

2912

In [23]:
cls[2912]

-0.068503216

In [28]:
cls[2912]

-0.068503216

In [29]:
len(cls)

5472

In [30]:
data['y_test']

<HDF5 dataset "y_test": shape (2913, 1), type "|i1">